# 🗺 Turkey: Raster-based market access


This notebook replicates an infrastructure analysis of DEC's [INFRA_SAP](https://github.com/worldbank/INFRA_SAP). The output raster maps will be manipulated on QGIS to generate the final results.

In [ ]:
# https://papermill.readthedocs.io/en/latest/usage-parameterize.html
FEATURE = "airports"

Input:

In [ ]:
DESTINATIONS = f"../data/BASE_NATIONAL_DATA/INFRASAP/TUR/{FEATURE}.shp"
FRICTION_SURFACE = "../data/tur-friction-surface.tif"

In [ ]:
inD = gpd.read_file(DESTINATIONS)
inR = rasterio.open(FRICTION_SURFACE)

frictionD = inR.read()[0,:,:]

In [ ]:
frictionD.shape

Convert friction surface to traversal time (lazily). Original data are minutes to travel 1m, so we will convert to minutes to cross the cell (1Km)

In [ ]:
mcp = graph.MCP_Geometric(frictionD)

## Travel Costs

In [ ]:
travel_costs, traceback = ma.calculate_travel_time(inR, mcp, inD)
travel_costs = travel_costs.astype(inR.meta['dtype'])

with rasterio.open(os.path.join("../data", "results", f"least_cost_travel_time_{FEATURE}.tif"), 'w', **inR.meta) as out_file:
    out_file.write_band(1, travel_costs)

## Marketsheds

In [ ]:
outfile = os.path.join("../data", "results", f"marketsheds_{FEATURE}.tif")
ma.generate_market_sheds(inR, inD, out_file=outfile)